In [72]:
from ultralytics import YOLO
from ultralytics.data.converter import convert_segment_masks_to_yolo_seg
import os
from PIL import Image
import torch
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [ ]:
model = YOLO("yolov8n.pt")
results = model.train(data="data.yaml", epochs=1, batch=8, imgsz=960)

In [ ]:
model.export(format="torchscript")

In [ ]:
images = []
for image_name in os.listdir("images/"):
    images.append(Image.open(os.path.join("images/", image_name)))

In [ ]:
model_torch = torch.jit.load("best_no_seg.torchscript")
model_torch.eval()

images_np = []
for image in images:
    images_np.append(np.transpose(np.array(image.resize((960, 960))), (2, 0, 1)) / 255.0)
images_tensor = torch.tensor(images_np).float()

results = model_torch.forward(images_tensor)

In [ ]:
results[0].shape

In [ ]:
results[1].shape

In [68]:
prototypes = torch.load("prototypes.pt").state_dict()["0"][0]
prototypes

tensor([[[ 4.5228e-01,  3.7155e-01,  3.6625e-01,  ...,  4.3602e-01,  4.3478e-01,  4.3795e-01],
         [ 6.2003e-01,  5.4518e-01,  5.7179e-01,  ...,  2.7773e-01,  3.5290e-01,  4.0929e-01],
         [ 7.6795e-01,  7.4576e-01,  5.8587e-01,  ...,  2.0646e-01,  3.0003e-01,  3.9186e-01],
         ...,
         [ 1.5988e+00,  3.2835e+00,  3.2877e+00,  ...,  2.0623e-01,  4.5414e-04,  8.0150e-02],
         [ 1.0783e+00,  1.5016e+00,  2.8702e+00,  ...,  1.4926e-01,  5.9504e-02,  1.2430e-01],
         [ 1.1034e+00,  1.3637e+00,  1.6933e+00,  ...,  2.4880e-01,  2.1729e-01,  2.8493e-01]],

        [[-2.2979e-01, -2.5511e-01, -2.4815e-01,  ..., -1.2564e-01, -1.2704e-01, -1.8223e-01],
         [-1.2855e-01, -1.4144e-01, -1.2132e-01,  ..., -2.4576e-01, -1.8096e-01, -1.8283e-01],
         [-6.3495e-03, -3.1821e-02, -1.4676e-01,  ..., -2.6636e-01, -2.3719e-01, -2.2832e-01],
         ...,
         [ 1.4265e+00,  4.6539e+00,  5.2600e+00,  ..., -2.4165e-01, -2.7360e-01, -2.7838e-01],
         [ 3.8605e-0

In [69]:
prototypes = np.array(prototypes.cpu())

In [71]:
num_images = len(prototypes)
num_cols = 8  # Change this according to your preference
num_rows = (num_images + num_cols - 1) // num_cols

# Plot the images
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 5))

for i, ax in enumerate(axes.flat):
    if i < num_images:
        ax.imshow(prototypes[i], cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')  # Hide axes for empty subplots

plt.tight_layout()
plt.show()

<Figure size 1000x500 with 32 Axes>